In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [19]:
import argparse
import yaml
import time
import datetime
import cv2
import numpy as np
import pandas as pd
import random

from skimage import io, transform
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
from scipy import ndimage, misc

import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms, utils
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
import torchvision
import torch
from torch.autograd import Variable
import torch.optim as optim
from sklearn.metrics import mean_absolute_error

from google.colab import drive
import os
import warnings
from collections import defaultdict
from sklearn.model_selection import KFold
import time
from scipy import stats
import sys


In [20]:
categorical_columns = ['TRAPTYPE', 'ATTRACTANTSUSED', 'SETTIMEOFDAY', 'COLLECTTIMEOFDAY', 'GENUS', 'SPECIES', 'TRAPID','TRAPSITE']
numerical_columns = ['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR','TRAPCOLLECT', 'DIFF_DAYS']

weather_columns = ['sunrise', 'sunset', 'moonrise', 
                     'moonset',  'moon_illumination', 'maxtempC', 'maxtempF',
                     'mintempC', 'mintempF', 'avgtempC', 'avgtempF',
                     'totalSnow_cm', 'sunHour', 'uvIndex', 'time', 'tempC', 'tempF', 'windspeedMiles', 
                     'windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM','precipInches', 'visibility',
                     'visibilityMiles', 'pressure', 'pressureInches', 'cloudcover','HeatIndexC', 'HeatIndexF',
                     'DewPointC','DewPointF', 'WindChillC', 'WindChillF', 'WindGustMiles','WindGustKmph', 'FeelsLikeC', 'FeelsLikeF']

for i in range(1,15):
    for item in weather_columns:
        numerical_columns.append(item + str(i))
    for item in ['moon_phase', 'winddir16Point']:
        categorical_columns.append(item + str(i))
print(numerical_columns)

['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR', 'TRAPCOLLECT', 'DIFF_DAYS', 'sunrise1', 'sunset1', 'moonrise1', 'moonset1', 'moon_illumination1', 'maxtempC1', 'maxtempF1', 'mintempC1', 'mintempF1', 'avgtempC1', 'avgtempF1', 'totalSnow_cm1', 'sunHour1', 'uvIndex1', 'time1', 'tempC1', 'tempF1', 'windspeedMiles1', 'windspeedKmph1', 'winddirDegree1', 'weatherCode1', 'precipMM1', 'precipInches1', 'visibility1', 'visibilityMiles1', 'pressure1', 'pressureInches1', 'cloudcover1', 'HeatIndexC1', 'HeatIndexF1', 'DewPointC1', 'DewPointF1', 'WindChillC1', 'WindChillF1', 'WindGustMiles1', 'WindGustKmph1', 'FeelsLikeC1', 'FeelsLikeF1', 'sunrise2', 'sunset2', 'moonrise2', 'moonset2', 'moon_illumination2', 'maxtempC2', 'maxtempF2', 'mintempC2', 'mintempF2', 'avgtempC2', 'avgtempF2', 'totalSnow_cm2', 'sunHour2', 'uvIndex2', 'time2', 'tempC2', 'tempF2', 'windspeedMiles2', 'windspeedKmph2', 'winddirDegree2', 'weatherCode2', 'precipMM2', 'precipInches2', 'visibility2', 'visibilityMiles2', 'pressure2', 'press

In [62]:
sentinel_frame = pd.read_csv('/content/drive/My Drive/BerkeleyResults/Approach2/original_mosquito_wwo.csv')


sentinel_frame["TRAPSET"]= pd.to_datetime(sentinel_frame["TRAPSET"])
sentinel_frame["TRAPCOLLECT"]= pd.to_datetime(sentinel_frame["TRAPCOLLECT"])
sentinel_frame["DIFF_DAYS"] = (sentinel_frame["TRAPCOLLECT"] - sentinel_frame["TRAPSET"]).dt.days
sentinel_frame["TRAPSET"]= sentinel_frame["TRAPSET"].dt.week
sentinel_frame["TRAPCOLLECT"]= sentinel_frame["TRAPCOLLECT"].dt.week

for category in categorical_columns:
    sentinel_frame[category] = sentinel_frame[category].astype('category')
    # print( sentinel_frame[category].dtype)
    # sentinel_frame[category] = pd.Categorical(sentinel_frame[category]).codes

categorical_column_sizes = [len(sentinel_frame[column].cat.categories) for column in categorical_columns]
categorical_embedding_sizes = [(col_size, min(50, (col_size+1)//2)) for col_size in categorical_column_sizes]

for col in numerical_columns:
    sentinel_frame[col] = (sentinel_frame[col] - sentinel_frame[col].mean()) / (sentinel_frame[col].max() - sentinel_frame[col].min())
    sentinel_frame[col] = sentinel_frame[col].fillna(0)
    
def get_categorical_tensor(in_frame, categories):
    categorical_data = np.stack([in_frame[col].cat.codes.values for col in categories], 1)
    categorical_data = torch.tensor(categorical_data, dtype=torch.int64)

    return categorical_data

def get_numerical_tensor(in_frame, numerical_columns):
    numerical_data = np.stack([in_frame[col].values for col in numerical_columns], 1)
    numerical_data = torch.tensor(numerical_data, dtype=torch.float)
    return numerical_data





In [63]:
# cor = sentinel_frame.corr(method='kendall')

# x = cor["TOTAL"].sort_values(ascending=False)
pd.set_option('display.max_rows', 99999)
print(sentinel_frame.columns[sentinel_frame.isna().any()].tolist()
)

[]


In [25]:
cor = sentinel_frame.corr(method='spearman')

x = cor["TOTAL"].sort_values(ascending=False)
pd.set_option('display.max_rows', len(x))

print(x)

TOTAL                  1.000000
sunHour13              0.199736
DIFF_DAYS              0.176961
sunset14               0.152823
sunset13               0.151717
sunset12               0.150126
sunHour6               0.149667
sunset11               0.146093
sunHour12              0.145931
sunset10               0.142539
sunset9                0.138954
sunHour14              0.137951
sunset8                0.136304
sunHour5               0.131206
sunset7                0.129701
sunset6                0.127237
WindGustMiles4         0.125458
sunset5                0.124297
WindGustKmph4          0.121924
sunset4                0.119754
sunset3                0.112654
OBJECTID               0.112038
sunset2                0.106846
WindGustMiles8         0.104682
WindGustMiles6         0.104250
windspeedMiles4        0.103778
WindGustKmph6          0.103404
windspeedKmph4         0.102885
sunset1                0.102463
WindGustKmph8          0.100848
winddirDegree5         0.098911
sunHour1

In [26]:
cor = sentinel_frame.corr(method='pearson')

x = cor["TOTAL"].sort_values(ascending=False)
pd.set_option('display.max_rows', len(x))

print(x)

TOTAL                  1.000000
sunHour13              0.187606
sunHour12              0.180901
sunHour6               0.178837
sunHour14              0.158396
sunHour5               0.141958
sunset14               0.140950
sunHour11              0.140111
sunset13               0.140051
sunset12               0.138997
sunset11               0.138046
sunset10               0.136041
sunset9                0.134668
sunHour7               0.133806
sunHour10              0.133566
sunset8                0.133458
sunset7                0.131645
sunset6                0.130140
sunset5                0.128254
sunset4                0.126325
sunHour9               0.125606
sunset3                0.124032
sunset2                0.121616
sunset1                0.119871
sunHour2               0.116599
sunHour3               0.114620
sunHour1               0.114192
sunHour4               0.113630
OBJECTID               0.111428
sunHour8               0.099993
winddirDegree13        0.094501
DIFF_DAY

In [64]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * torch.sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_tensors[0]
        sigmoid_i = torch.sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))
    
class SwishM(nn.Module):
    def forward(self, input_tensor):
        return Swish.apply(input_tensor)
        
def create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout):
    all_layers = []
    num_categorical_cols = sum((nf for ni, nf in embedding_size))
    input_size = num_categorical_cols + num_numerical_cols

    all_layers.append(nn.Linear(input_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, output_size))
    all_layers.append(nn.ReLU())

    return nn.Sequential(*all_layers)

class Sentinel_net(nn.Module):

    def __init__(self, embedding_size, num_numerical_cols, output_size, dropout, hidden_size):
        super(Sentinel_net, self).__init__()
        self.layers = create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout)
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        self.embedding_dropout = nn.Dropout(dropout)
        self.batch_norm_num = nn.BatchNorm1d(num_numerical_cols)

    def forward(self, x_categorical, x_numerical):

        embeddings = []
        for i, e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:, i]))
        y = torch.cat(embeddings, 1)
        y = self.embedding_dropout(y)
        x_numerical = self.batch_norm_num(x_numerical)
        y = torch.cat([y, x_numerical], 1)
        y = self.layers(y)

        return y

In [65]:
def train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr):
    model = Sentinel_net(categorical_embedding_sizes, train_n_t.shape[1], 1, dropout, hidden_size)
    model.train().cuda()

    criterion = torch.nn.MSELoss(reduction='mean') 
    params_dict = dict(model.named_parameters())
    params = []
    best = sys.maxsize
    r_value = 0
    std_err = 0
    
    for key, value in params_dict.items():
        if 'weight' in key and 'layers' in key:
            params += [{'params':value, 'weight_decay':4e-3}]
        else:
            params += [{'params':value, 'weight_decay':0.0}]

    optimizer = torch.optim.RMSprop(params, lr=lr, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0.9, centered=False)
    epoch = 0

    for i in range(1, 1000):
        y_pred = model(train_c_t, train_n_t)
        loss = criterion(y_pred.squeeze(), outputs_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            y_pred2 = model(test_c_t, test_n_t)
            c_out, c_pred = outputs_test.cpu().detach().numpy(), y_pred2.cpu().detach().numpy().squeeze()
            current_score = mean_absolute_error(c_out, c_pred)
            if current_score < best:
                best = current_score
                m, b, r_value, p_value, std_err = stats.linregress(c_out,c_pred)

            model.train()
    
    #print('MAE = ',current_score, 'r2 = ',r_value, 'SE = ',std_err)
    return best, r_value, std_err

In [66]:
start_time = time.time()
learning = [0.01, 0.001, 0.0001] 
best_score_k_fold = sys.maxsize

print('2 Layers Architecture')

for dropout in range(0,7):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(50, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame.iloc[train_index], sentinel_frame.iloc[test_index]

                train_c_t = get_categorical_tensor(train, categorical_columns).cuda()
                test_c_t = get_categorical_tensor(test, categorical_columns).cuda()
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

2 Layers Architecture
MAE =  10.677099609375 r2 =  0.44812129047967836 SE =  0.024689154242104806 lr =  0.01 hz =  50 dr =  0.0
Best so far 10.677099609375

MAE =  10.9135892868042 r2 =  0.41556095789587727 SE =  0.01869905534464378 lr =  0.001 hz =  50 dr =  0.0
MAE =  12.431002426147462 r2 =  0.3795804488179614 SE =  0.012304522037975671 lr =  0.0001 hz =  50 dr =  0.0
MAE =  10.920509719848633 r2 =  0.42186804550065393 SE =  0.024438309367472443 lr =  0.01 hz =  60 dr =  0.0
MAE =  11.0544677734375 r2 =  0.36904784789376843 SE =  0.022641444477506052 lr =  0.001 hz =  60 dr =  0.0
MAE =  12.690280723571778 r2 =  0.34313742550179616 SE =  0.01984994145165109 lr =  0.0001 hz =  60 dr =  0.0
MAE =  11.189033508300781 r2 =  0.3846112967519658 SE =  0.025860811884398406 lr =  0.01 hz =  70 dr =  0.0
MAE =  11.730233001708985 r2 =  0.3778102437111661 SE =  0.019843135551241613 lr =  0.001 hz =  70 dr =  0.0
MAE =  12.928387069702149 r2 =  0.34818495269747807 SE =  0.014286710238692052 lr 